In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.6 MB/s eta 0:00:00


In [2]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c forest-type-classification-spai

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ayushparwal2026/country-population-from-1960-to-2022            Country Population from 1960 to 2022                75KB  2024-05-31 08:58:26           1159         28  0.9411765        
devarajv88/walmart-sales-dataset                                e-Commerce (Walmart) Sales Dataset                   5MB  2024-05-30 12:09:55           1436         26  1.0              
shreyanshverma27/online-sales-dataset-popular-marketplace-data  Online Sales Dataset - Popular Marketplace Data      7KB  2024-05-25 23:55:26           2359         39  1.0              
kanchana1990/vehicle-dataset-2024                               V

In [3]:
!unzip /content/forest-type-classification-spai.zip

Archive:  /content/forest-type-classification-spai.zip
  inflating: metaData.csv            
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import optuna

In [5]:
import pandas as pd
import lightgbm as lgb
import time
from sklearn.model_selection import cross_val_score

In [6]:
def NDVI(row):
    ndvi = (row['08NIR'] - row['04Red']) / (row['08NIR'] + row['04Red'])
    return ndvi
def EVI(row):
    evi  = (row['08NIR'] - row['04Red'])/((row['08NIR']+6*row['04Red']-7.5*row['02Blue'])+1)
    return evi
def CVI(row):
    cvi = row['08NIR']*(row['04Red']/(row['03Green'])**2)
    return cvi
def NBR(row):
    nbr = (row['08NIR'] - row['13SWIR2'])/(row['08NIR'] + row['13SWIR2'])
    return nbr
def ARI(row):
    ari = (1/row['03Green'])-(1/row['04Red'])
    return ari
def TVI(row):
    tvi = 0.5 * (120 * (row['08NIR'] - row['03Green']) - 200 * (row['04Red'] - row['03Green']))
    return tvi
def VARI(row):
    vari = (row['03Green']-row['04Red'])/(row['03Green']+row['04Red']-row['02Blue'])
    return vari
def NDRE(row):
    ndre = (row['08NIR'] - row['05Veg5']) / (row['08NIR'] + row['05Veg5'])
    return ndre
def RDVI(row):
    rdvi = (row['08NIR'] - row['04Red']) / ((row['08NIR'] + row['04Red']) ** 0.5)
    return rdvi
def GLI(row):
    gli = (2 * row['03Green'] - row['04Red'] - row['02Blue']) / (2 * row['03Green'] + row['04Red'] + row['02Blue'])
    return gli

In [7]:
# test.csv
train_df = pd.read_csv("/content/train.csv")
# train_df = train_df[0:10]
# train_df
train_df = train_df.rename(columns = {
    'b1':'01Coastal',
    'b2':'02Blue',
    'b3':'03Green',
    'b4':'04Red',
    'b5':'05Veg5',
    'b6':'06Veg6',
    'b7':'07Veg7',
    'b8':'08NIR',
    'b8_a':'09NIRn',
    'b9':'10WV',
    'b10':'11SWIR-C',
    'b11':'12SWIR1',
    'b12':'13SWIR2'
})
train_df['NDVI'] = train_df.apply(NDVI, axis=1)
train_df['EVI']  = train_df.apply(EVI, axis=1)
train_df['CVI']  = train_df.apply(CVI, axis=1)
train_df['NBR']  = train_df.apply(NBR, axis=1)
train_df['ARI']  = train_df.apply(ARI, axis=1)
train_df['TVI']  = train_df.apply(TVI, axis=1)
train_df['VARI'] = train_df.apply(VARI, axis=1)
train_df['NDRE'] = train_df.apply(NDRE, axis=1)
train_df['RDVI'] = train_df.apply(VARI, axis=1)
train_df['GLI']  = train_df.apply(NDRE, axis=1)

Important_11_feature = ['id','12SWIR1','10WV','13SWIR2','09NIRn','01Coastal','05Veg5','CVI','NDRE','NBR','02Blue','nforest_type']
train_df = train_df[Important_11_feature]
train_df

,id,12SWIR1,10WV,13SWIR2,09NIRn,01Coastal,05Veg5,CVI,NDRE,NBR,02Blue,nforest_type
0,2002,1927,3039,1038,2343,293,987,3.814737,0.316245,0.293397,278,MDF
1,3212,1598,2690,697,2685,197,682,4.268053,0.535422,0.527618,201,DDF
2,13312,1975,2683,1031,2671,929,1220,1.968043,0.324474,0.397604,982,MDF
3,17020,1560,2955,689,3005,132,609,3.179066,0.664740,0.628872,189,MDF
4,5967,1944,2043,1131,2077,241,918,2.863677,0.299237,0.201553,362,MDF
...,...,...,...,...,...,...,...,...,...,...,...,...
13048,9185,1940,2100,1054,2291,374,977,3.290099,0.366818,0.333544,382,DDF
13049,13977,3602,6053,2720,4097,1983,2314,2.182249,0.258096,0.181216,1622,DDF
13050,755,2007,2170,1148,2247,940,1252,1.611674,0.216030,0.256958,975,DDF
13051,1616,2312,3380,1190,3396,1174,1310,2.138577,0.399083,0.438679,1112,MDF


In [8]:
test_df = pd.read_csv("/content/test.csv")
# test_df = test_df[0:10]
test_df = test_df.rename(columns = {
    'b1':'01Coastal',
    'b2':'02Blue',
    'b3':'03Green',
    'b4':'04Red',
    'b5':'05Veg5',
    'b6':'06Veg6',
    'b7':'07Veg7',
    'b8':'08NIR',
    'b8_a':'09NIRn',
    'b9':'10WV',
    'b10':'11SWIR-C',
    'b11':'12SWIR1',
    'b12':'13SWIR2'
})
test_df['NDVI'] = test_df.apply(NDVI, axis=1)
test_df['EVI']  = test_df.apply(EVI, axis=1)
test_df['CVI']  = test_df.apply(CVI, axis=1)
test_df['NBR']  = test_df.apply(NBR, axis=1)
test_df['ARI']  = test_df.apply(ARI, axis=1)
test_df['TVI']  = test_df.apply(TVI, axis=1)
test_df['VARI'] = test_df.apply(VARI, axis=1)
test_df['NDRE'] = test_df.apply(NDRE, axis=1)
test_df['RDVI'] = test_df.apply(VARI, axis=1)
test_df['GLI']  = test_df.apply(NDRE, axis=1)

Important_10_feature = ['id','12SWIR1','10WV','13SWIR2','09NIRn','01Coastal','05Veg5','CVI','NDRE','NBR','02Blue']
test_df = test_df[Important_10_feature]
test_df

,id,12SWIR1,10WV,13SWIR2,09NIRn,01Coastal,05Veg5,CVI,NDRE,NBR,02Blue
0,13467,1425,2595,693,2611,69,847,3.256395,0.474729,0.548681,312
1,12719,1514,2582,691,2359,242,718,2.939358,0.549843,0.563073,343
2,1054,2354,3149,1118,3645,218,965,3.890900,0.554889,0.501894,292
3,13747,2013,2345,1134,2290,350,982,3.302805,0.396991,0.334702,306
4,9453,1450,2193,712,2200,185,673,4.389421,0.533611,0.513162,293
...,...,...,...,...,...,...,...,...,...,...,...
3995,115,1686,3312,811,3126,447,958,2.993690,0.511723,0.570559,425
3996,10654,2694,2856,1503,3212,252,1294,3.332794,0.348603,0.281205,470
3997,5718,1486,3087,618,3086,233,699,4.709441,0.625101,0.661184,249
3998,13054,1840,3161,774,3762,221,703,3.900659,0.647354,0.618531,245


In [9]:
print("train data shape:",train_df.shape)
print("test data shape :",test_df.shape)

train data shape: (13053, 12)
test data shape : (4000, 11)


In [10]:
X = train_df.drop(columns=['nforest_type','id'])
y = train_df['nforest_type']
X_test  = test_df

<hr>

In [11]:
start = time.time()
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'num_leaves': trial.suggest_int('num_leaves', 31, 2047),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        # 'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        # 'device_type': 'gpu',  # Ensure GPU is used if required
        # Additional parameters
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),  # L2 regularization
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0)  # Useful for imbalanced datasets
    }

    clf = lgb.LGBMClassifier(
        **params
    )

    return cross_val_score(
        clf, X, y, n_jobs=-1, cv=3
    ).mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
end   = time.time()


[I 2024-06-04 16:48:02,232] A new study created in memory with name: no-name-7c77fbea-d98e-4c95-9210-2944ef0847be
[I 2024-06-04 16:48:31,100] Trial 0 finished with value: 0.6711100896345666 and parameters: {'n_estimators': 482, 'learning_rate': 0.008278172365435094, 'max_depth': 16, 'num_leaves': 1817, 'min_child_samples': 65, 'feature_fraction': 0.6866494741189693, 'bagging_fraction': 0.9454249088267708, 'bagging_freq': 7, 'lambda_l1': 2.3284250904264957e-07, 'lambda_l2': 3.615402252780299e-05, 'min_split_gain': 0.10751875805861477, 'subsample': 0.5793012140068403, 'colsample_bytree': 0.8906573899009507, 'subsample_for_bin': 293147, 'reg_alpha': 0.0010289762456566277, 'reg_lambda': 3.111460752934331e-05, 'scale_pos_weight': 8.021241414062441}. Best is trial 0 with value: 0.6711100896345666.
[I 2024-06-04 16:48:44,402] Trial 1 finished with value: 0.6486631425725887 and parameters: {'n_estimators': 644, 'learning_rate': 0.002918906579161455, 'max_depth': 14, 'num_leaves': 430, 'min_chi

In [19]:
print((end-start)/60)
trial = study.best_trial
print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
best_params = trial.params
best_params

53.120737834771475
Accuracy: 0.6795372711254117
Best hyperparameters: {'n_estimators': 1265, 'learning_rate': 0.004278557786049502, 'max_depth': 11, 'num_leaves': 1667, 'min_child_samples': 25, 'feature_fraction': 0.9645974662724492, 'bagging_fraction': 0.6748015940634287, 'bagging_freq': 1, 'lambda_l1': 9.517434376405417e-06, 'lambda_l2': 1.0141290904972833e-08, 'min_split_gain': 0.49523874126762507, 'subsample': 0.5354065908623218, 'colsample_bytree': 0.9952931155691193, 'subsample_for_bin': 151130, 'reg_alpha': 2.621609579874895e-06, 'reg_lambda': 1.605702436924837e-05, 'scale_pos_weight': 9.468280123334706}


{'n_estimators': 1265,
 'learning_rate': 0.004278557786049502,
 'max_depth': 11,
 'num_leaves': 1667,
 'min_child_samples': 25,
 'feature_fraction': 0.9645974662724492,
 'bagging_fraction': 0.6748015940634287,
 'bagging_freq': 1,
 'lambda_l1': 9.517434376405417e-06,
 'lambda_l2': 1.0141290904972833e-08,
 'min_split_gain': 0.49523874126762507,
 'subsample': 0.5354065908623218,
 'colsample_bytree': 0.9952931155691193,
 'subsample_for_bin': 151130,
 'reg_alpha': 2.621609579874895e-06,
 'reg_lambda': 1.605702436924837e-05,
 'scale_pos_weight': 9.468280123334706}

In [20]:
final_clf = lgb.LGBMClassifier(
    **best_params,
    # device_type = 'GPU'
)
final_clf.fit(X,y)

[LightGBM] [Warning] feature_fraction is set=0.9645974662724492, colsample_bytree=0.9952931155691193 will be ignored. Current value: feature_fraction=0.9645974662724492
[LightGBM] [Warning] lambda_l2 is set=1.0141290904972833e-08, reg_lambda=1.605702436924837e-05 will be ignored. Current value: lambda_l2=1.0141290904972833e-08
[LightGBM] [Warning] lambda_l1 is set=9.517434376405417e-06, reg_alpha=2.621609579874895e-06 will be ignored. Current value: lambda_l1=9.517434376405417e-06
[LightGBM] [Warning] bagging_fraction is set=0.6748015940634287, subsample=0.5354065908623218 will be ignored. Current value: bagging_fraction=0.6748015940634287
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9645974662724492, colsample_bytree=0.9952931155691193 will be ignored. Current value: feature_fraction=0.9645974662724492
[LightGBM] [Warning] lambda_l2 is set=1.0141290904972833e-08, reg_lambda=1.

LGBMClassifier(bagging_fraction=0.6748015940634287, bagging_freq=1,
               colsample_bytree=0.9952931155691193,
               feature_fraction=0.9645974662724492,
               lambda_l1=9.517434376405417e-06,
               lambda_l2=1.0141290904972833e-08,
               learning_rate=0.004278557786049502, max_depth=11,
               min_child_samples=25, min_split_gain=0.49523874126762507,
               n_estimators=1265, num_leaves=1667,
               reg_alpha=2.621609579874895e-06,
               reg_lambda=1.605702436924837e-05,
               scale_pos_weight=9.468280123334706, subsample=0.5354065908623218,
               subsample_for_bin=151130)

In [21]:
final_clf.predict(X_test[X_test['id'] == 13467].drop(columns=['id']))

[LightGBM] [Warning] feature_fraction is set=0.9645974662724492, colsample_bytree=0.9952931155691193 will be ignored. Current value: feature_fraction=0.9645974662724492
[LightGBM] [Warning] lambda_l2 is set=1.0141290904972833e-08, reg_lambda=1.605702436924837e-05 will be ignored. Current value: lambda_l2=1.0141290904972833e-08
[LightGBM] [Warning] lambda_l1 is set=9.517434376405417e-06, reg_alpha=2.621609579874895e-06 will be ignored. Current value: lambda_l1=9.517434376405417e-06
[LightGBM] [Warning] bagging_fraction is set=0.6748015940634287, subsample=0.5354065908623218 will be ignored. Current value: bagging_fraction=0.6748015940634287
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


array(['DEF'], dtype=object)

In [22]:
df = pd.read_csv("/content/sample_submission.csv")
for idx,row in df.iterrows():
#     print(X_test[X_test['id'] == row['id']])
    data = X_test[X_test['id'] == row['id']].drop(columns=['id'])      # เปิดข้อมูลรายตัวตาม id ของ sample_submission.csv file
    result = final_clf.predict(data)
    df.at[idx,'nforest_type'] = result
#     break

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] feature_fraction is set=0.9645974662724492, colsample_bytree=0.9952931155691193 will be ignored. Current value: feature_fraction=0.9645974662724492
[LightGBM] [Warning] lambda_l2 is set=1.0141290904972833e-08, reg_lambda=1.605702436924837e-05 will be ignored. Current value: lambda_l2=1.0141290904972833e-08
[LightGBM] [Warning] lambda_l1 is set=9.517434376405417e-06, reg_alpha=2.621609579874895e-06 will be ignored. Current value: lambda_l1=9.517434376405417e-06
[LightGBM] [Warning] bagging_fraction is set=0.6748015940634287, subsample=0.5354065908623218 will be ignored. Current value: bagging_fraction=0.6748015940634287
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9645974662724492, colsample_bytree=0.9952931155691193 will be ignored. Current value: feature_fraction=0.9645974662724492
[LightGBM] [Warning] lam

In [16]:
df

,id,nforest_type
0,13467,DEF
1,12719,DDF
2,1054,MDF
3,13747,DDF
4,9453,DDF
...,...,...
3995,115,DEF
3996,10654,MDF
3997,5718,DDF
3998,13054,MDF


In [17]:
version = '0.2.0'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
df.to_csv(csv_name,index=False)

v.0.2.0.2441.csv
